In [1]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from tqdm import tqdm
import time
import itertools
import imutils
import seaborn as sns
import urllib
from sklearn.preprocessing import LabelEncoder
from networkx.algorithms.clique import enumerate_all_cliques
from networkx.algorithms.clique import cliques_containing_node

+ MILK
+ ALMOND
+ CHOCOLATE

In [ ]:
#!pip install google_images_download

In [ ]:
from google_images_download import google_images_download  
response = google_images_download.googleimagesdownload()  
  
search_queries = ['almond milk']


In [ ]:
def downloadimages(query): 
    # keywords is the search query 
    # format is the image file format 
    # limit is the number of images to be downloaded 
    # print urs is to print the image file url 
    # size is the image size which can 
    # be specified manually ("large, medium, icon") 
    # aspect ratio denotes the height width ratio 
    # of images to download. ("tall, square, wide, panoramic") 
    arguments = {"keywords": query, 
                 "format": "jpg", 
                 "limit":4, 
                 "print_urls":True, 
                 "size": "medium", 
                 "aspect_ratio": "panoramic"}
    try: 
        response.download(arguments) 
      
    # Handling File NotFound Error     
    except FileNotFoundError:  
        arguments = {"keywords": query, 
                     "format": "jpg", 
                     "limit":2, 
                     "print_urls":True,  
                     "size": "medium"} 
                       
        # Providing arguments for the searched query 
        try: 
            # Downloading the photos based 
            # on the given arguments 
            response.download(arguments)  
        except: 
            pass

In [ ]:
query = 'almond milk'

arguments = {"keywords": query, 
             "format": "jpg", 
              "limit":1, 
              "print_urls":True, 
              "size": "medium", 
              "aspect_ratio": "panoramic"}

In [ ]:
def get_response(query):
    
    arguments = {"keywords": query, 
             "format": "jpg", 
              "limit":1, 
              "print_urls":True, 
              "size": "medium", 
              "aspect_ratio": "panoramic"}
    
    output = response.download(arguments) 
    
    return output[0][query][0]
    

In [ ]:
item1 = 'almond'
item2 = 'milk'

In [ ]:
img_alm = get_response(item1)

In [ ]:
img_mlk = get_response(item2)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Input
from keras import applications
from keras.preprocessing import image
from keras.layers import Dense
from keras.applications.mobilenet import MobileNet
import cv2
from scipy.spatial.distance import cosine as cs


In [ ]:
#### Downloading the Pre-trained Model
mobnet_model = keras.applications.mobilenet.MobileNet(input_shape=(224,224,3),include_top=False,pooling='max')

In [ ]:
inc_model = keras.applications.inception_v3.InceptionV3(input_shape=(None,None,3),include_top=False)

In [ ]:
def get_feat_image(img): 
    resized_image = cv2.resize(img,(224,224))
    return resized_image


#Extracting the PRE-Trained Embedding

def get_deep_features(img):
    
    img = get_feat_image(img)
    array= image.img_to_array(img)
    arr_batch = np.expand_dims(array,axis=0)
    arr_prep = keras.applications.mobilenet.preprocess_input(arr_batch)
    feat = mobnet_model.predict(arr_prep)
        
    return feat

In [ ]:
#image embedding
query_alm_milk = cv2.imread('/Users/s0c02nj/downloads/almond milk/1.almond-milk-960x480.jpg') 
query_embed = get_deep_features(query_alm_milk)

In [ ]:
item_mlk_choc = cv2.imread('/Users/s0c02nj/Desktop/milk_chocolate_coating.jpeg') 
item_mlk_choc_emb =  get_deep_features(item_mlk_choc)

In [ ]:
item_mlk = cv2.imread('/Users/s0c02nj/Desktop/milk.jpeg') 
item_mlk_emb =  get_deep_features(item_mlk)

In [ ]:
chocolate = cv2.imread('/Users/s0c02nj/Desktop/chocolate.jpeg') 
choc_emb = get_deep_features(chocolate)


In [ ]:
#image embedding
item_potato = cv2.imread('/Users/s0c02nj/Desktop/potato.jpeg') 
item_potato_embed = get_deep_features(item_potato)

In [ ]:
#image embedding
item_baked_potato = cv2.imread('/Users/s0c02nj/Desktop/baked_potato.jpeg') 
item_baked_potato_embed = get_deep_features(item_baked_potato)

In [ ]:
#image embedding
item_potato_flour = cv2.imread('/Users/s0c02nj/Desktop/potato_flour.jpeg') 
item_potato_flour_embed = get_deep_features(item_potato_flour)

In [ ]:
score = 1 - cs(item_potato_flour_embed,item_potato_embed)
score

In [ ]:
score = 1 - cs(item_baked_potato_embed,item_potato_embed)
score

In [ ]:
score = 1 - cs(item_mlk_choc_emb,item_mlk_emb)
score

In [ ]:
score = 1 - cs(item_mlk_choc_emb,choc_emb)
score

In [ ]:
#Finding the cosine similarity of the window and query features
score_q_milk  = 1 - cs(item_mlk_emb,query_embed)
score_q_milk

In [ ]:
#Finding the cosine similarity of the window and query features
score_q_alm  = 1 - cs(item_alm_emb,query_embed)
score_q_alm

#### Ingredient Clustering

In [2]:
df_prim = pd.read_csv('/Users/s0c02nj/Downloads/primary_ing_share_filt.csv')

In [3]:
df_prim['ind'] = df_prim['Primary Ingredient'].apply(lambda x: 1 if len(x.split())<= 4 else 0)

In [4]:
#df_prim[df_prim['ind'] == 0].shape
#df_prim
df_prim_filt = df_prim[df_prim['ind'] == 1]
df_prim_filt.shape
#ingred_list = list(df_prim_filt['Primary Ingredient'].unique())

(51, 2)

In [5]:
#Expanding the data to form the graph
df1 = df_prim_filt[['Primary Ingredient']]

In [6]:
df1.head()

,Primary Ingredient
0,MILK
1,LOW FAT MILK
2,VITAMIN D3 MILK
3,LOW FATMILK
4,NON FAT MILK


In [7]:
le = LabelEncoder()
le.fit(df1['Primary Ingredient'])

LabelEncoder()

In [8]:
ingred_list = list(df1['Primary Ingredient'].unique())
new_list = list(itertools.combinations(ingred_list, 2))


In [9]:
df_total = pd.DataFrame(new_list)
df_total.columns = ['node1','node2']

In [10]:
jacc_ind = []

for i in tqdm(range(0,len(df_total))):
    
    node1 = df_total['node1'].iloc[i]
    node2 = df_total['node2'].iloc[i]
    
    s1 = set(node1.split())
    s2 = set(node2.split())
             
    
    if len(s1.intersection(s2)) > 0 :
        jacc_ind.append(1)
        
    else :
        jacc_ind.append(0)

100%|██████████| 1275/1275 [00:00<00:00, 35971.25it/s]


In [11]:
df_total['jacc_ind'] = jacc_ind

In [12]:
list_node1 = list(df_total['node1'])
node1_enc = list(le.transform(list_node1))

In [13]:
list_node2 = list(df_total['node2'])
node2_enc = list(le.transform(list_node2))

In [14]:
df_total['node1_enc'] = node1_enc

In [15]:
df_total['node2_enc'] = node2_enc

In [16]:
df_graph = df_total[['node1_enc','node2_enc','jacc_ind']]

In [17]:
df_graph_filt = df_graph[df_graph['jacc_ind'] == 1]

edges = []
for i in tqdm(range(0,len(df_graph_filt))) :
    edges.append((df_graph_filt['node1_enc'].iloc[i], df_graph_filt['node2_enc'].iloc[i]))

100%|██████████| 572/572 [00:00<00:00, 38490.35it/s]


#### Graph-based Solution

In [27]:
import igraph

In [25]:
#df_graph_filt.head(2)
#G = igraph.Graph()
#G.add_nodes_from(total_nodes_enc)

# add edges from txt 
#G.add_edges_from(edges)

##### Networkx ----- slow graph


In [ ]:
import networkx as nx

In [ ]:
#Getting all nodes
total_nodes_ls = list(df1['Primary Ingredient'])
total_nodes_enc = list(le.transform(total_nodes_ls))

In [ ]:
#total_nodes_enc = [1,2,3,4,5]

In [ ]:
#edges = [(1,2),(2,3),(1,3),(3,4),(4,5)]

In [ ]:
#df_graph_filt.head(2)
G = nx.Graph()
G.add_nodes_from(total_nodes_enc)

# add edges from txt 
G.add_edges_from(edges)

In [ ]:
list_cliques = list(enumerate_all_cliques(G))
#pre-process ---  remove digits

In [ ]:
len(list_cliques)

In [ ]:
len(list1)

In [ ]:
list1[20000000]

In [ ]:
list1 = []

for s in tqdm(list_cliques):
    if len(s) > 1 :
        list1.append(s)

In [ ]:
df1['Primary Ingredient'].iloc[40]

In [ ]:
#list2  = list(filter(lambda f: not any(set(f) < set(g) for g in list1), list1))

In [ ]:
#list1[1000000]

In [ ]:
list1[200]

In [ ]:
#le.transform(['MILK ALMOND'])
list1 = [s for s in list_cliques if len(s) > 2]

In [ ]:
#list1.sort(lambda x,y: len(x) < len(y))
list2 = sorted(list1, key=len)

In [ ]:
#list2[43]

In [ ]:
le.inverse_transform(list2[101])

In [ ]:
le.inverse_transform(list2[100])

In [ ]:
#59
#list1 = cliques_containing_node(G, nodes= 59,)

In [ ]:
?cliques_containing_node

In [ ]:
len(list1)

In [ ]:
le.inverse_transform(list1[2])

In [ ]:
le.inverse_transform(list1[10])

In [ ]:
#le.inverse_transform(list1[11])

In [ ]:
#len(set(list1[1]) - set(list1[13]))

In [ ]:
le.inverse_transform(total_nodes_enc)